In [1]:
import numpy    # for arrays
import time     # for timing cells

In [2]:
data_dir = 'C:/Users/navne/Python Files/Data Acquisition BIA 627/HW5/'
file_name_test = data_dir + 'testTrack_hierarchy.txt'
file_name_train = data_dir + 'trainIdx2_matrix.txt'
output_file = data_dir + 'output1.txt'

In [3]:
# Open files
f_test = open(file_name_test)
f_train = open(file_name_train)
train_line = f_train.readline()
f_out = open(output_file, 'w')

In [4]:
# Initialize arrays to hold track_id, album_id, and artist_id for all 6 entries of a user.
track_id_vec = [0] * 6
album_id_vec = [0] * 6
artist_id_vec = [0] * 6
last_user_id = -1

In [5]:
# Initialize numpy array to hold album and artist ratings for each user
user_rating_in_train = numpy.zeros(shape=(6,3))

In Part 1 of the process, we iterate over each line in the test file. Initially, we parse the line into a list and extract the `user_id`, `track_id`, `album_id`, and `artist_id`.

When we encounter a new user, indicated by a change in `user_id` (with `last_user_id` initially set to -1 to ensure we process the first user), we prepare vectors to store the ids for tracks, albums, and artists associated with this user.

As we transition to a new user in the test data, we begin examining the training data line by line. For each line, we construct a list from the data, capturing the `user_id`, `item_id`, and `rating`.

Should the `user_id` in the training data be lower than the `user_id` from the test data, we skip to the next iteration of the loop without further processing.

If the `user_id` in the training data matches the `user_id` from the test data, we look for matches between the album and artist ids in both the test and training data. When matches are found, we update the `user_rating_in_train` vector with the corresponding scores from the training data.

Upon finding a `user_id` in the training data that surpasses the `user_id` in the test data, it signals the start of data for a new user. At this point, we output six records for the current user, incorporating the `user_id`, `track_id`, and any discovered `album_score` and `artist_score` from the training data (defaulting to 0 if not found). After processing these six entries, we exit the loop to proceed with the next user.

In [6]:
start_time = time.time()

for line in f_test:
    arr_test = line.strip().split('|')
    user_id = arr_test[0]    # set user
    track_id = arr_test[1]   # set track
    album_id = arr_test[2]   # set album
    artist_id = arr_test[3]  # set artist
    
    if user_id != last_user_id:    # if new user reached
        ii = 0                     # reset ii
        user_rating_in_train = numpy.zeros(shape=(6,3))    # reset this array
        
    track_id_vec[ii] = track_id
    album_id_vec[ii] = album_id
    artist_id_vec[ii] = artist_id
    ii = ii + 1
    last_user_id = user_id
    
    if ii == 6:                    # if last entry for current user reached
        while(train_line):
            #for train_line in f_train:
                arr_train = train_line.strip().split('|')    # make list of user_id, item_id, and rating from current line
                train_user_id = arr_train[0]
                train_item_id = arr_train[1]
                train_rating = arr_train[2]
                train_line = f_train.readline()    # go to next line

                if train_user_id < user_id:
                    continue    # return to beginning of while loop
                if train_user_id == user_id:
                    for nn in range(0, 6):    # go through each row in (6,3) user_rating_in_train array for current row in training file
                        if train_item_id == album_id_vec[nn]:    # if album found
                            user_rating_in_train[nn, 0] = train_rating    # set album rating
                        if train_item_id == artist_id_vec[nn]:   # if artist found
                            user_rating_in_train[nn, 1] = train_rating    # set artist rating
                if train_user_id > user_id:   # if we reached the end of the current user, write to output file all the ratings found for current user
                    for nn in range(0, 6):    # go through each row in (6,3) array
                        out_str = str(user_id) + '|' + str(track_id_vec[nn]) + '|' + str(user_rating_in_train[nn,0]) + '|' + str(user_rating_in_train[nn, 1])
                        f_out.write(out_str + '\n')
                    break    # break out of while loop
                    
end_time = time.time()
elapsed_time = end_time - start_time
print(f'Done! Time elapsed - {elapsed_time:.2f} seconds.')

Done! Time elapsed - 24.50 seconds.


In [7]:
# Close all the files
f_test.close()
f_train.close()
f_out.close()

### Part 2 - Generating Prediction File

In this section, we utilize the data generated in Part 1 to compile a prediction file. This file will contain predictions for each track, formatted as `userID_trackID,prediction`. The prediction value will be either 1 or 0, where 1 indicates a prediction that the user will like the song, and 0 indicates a prediction that the user will not. Each user will have predictions for 6 tracks, consisting of 3 predictions marked as 1 and 3 as 0.

The prediction mechanism involves analyzing the album and artist ratings for each of the 6 tracks associated with a user. Album scores are given greater importance through higher weighting, under the assumption that a user is more likely to enjoy another song from the same album than from the same artist. The top 3 rated songs, based on these weighted scores, are predicted to be liked by the user (marked as 1), while the remaining 3 are predicted to be less favored (marked as 0).

In [8]:
# Initialize some values
ratings_array = numpy.zeros(shape=(2,6))
last_user_id = -1
track_id_out_vec = [0] * 6

start_time = time.time()

# Open files
output_file_2 = data_dir + 'Predictions.csv'
f_out_2 = open(output_file_2, 'w')
f_out = open(output_file)

# Write header
f_out_2.write('TrackID,Predictor\n')

# Go through each line of the output file
for line in f_out:
    arr_out = line.strip().split('|')
    user_id_out = arr_out[0]                # set user
    track_id_out = arr_out[1]               # set track
    album_rating = float(arr_out[2]) * .9   # set album and apply weight
    artist_rating = float(arr_out[3]) * .1  # set artist and apply weight
    
    if user_id_out != last_user_id:    # if new user reached
        ii = 0                         # reset ii
        ratings_array = numpy.zeros(shape=(2,6))    # reset this array
        
    if ii < 6:                                 # if still on same user (so from 0 to 5 inclusive)
        ratings_array[0, ii] = album_rating    # add album rating to this vector
        ratings_array[1, ii] = artist_rating   # add artist rating to this vector
        track_id_out_vec[ii] = track_id_out    # add track_id to this vector
        
    ii = ii + 1
    last_user_id = user_id_out
        
    if ii == 6:                                                    # if last entry for current user reached
        ratings_array_sum = numpy.zeros(shape=(6))                 # initialize numpy array for ratings sums
        numpy.sum(ratings_array, axis=0, out=ratings_array_sum)    # sum the album rating and artist rating for each track
        indices = (-ratings_array_sum).argsort()[:3]               # get indices of top 3 scores
        
        # Here we set the predictions by setting a 1 for elements with indices in the indices list and setting a 0 for elements with indices not in the indices list
        predictions = numpy.zeros(shape=(6))    # initlize numpy array for predictions
        for index in indices:
            predictions[index] = 1    
            
        # Here we write to the output file for the 6 track predictions for the current user
        for nn in range(0, 6):
            out_str = str(user_id_out) + '_' + str(track_id_out_vec[nn]) + ',' + str(int(predictions[nn]))
            f_out_2.write(out_str + '\n')

end_time = time.time()
elapsed_time = end_time - start_time
print(f'Done! Time elapsed - {elapsed_time:.2f} seconds.')

f_out_2.close()
f_out.close()

Done! Time elapsed - 0.86 seconds.


In [14]:
import numpy as np
import csv

def calculate_stats(ratings):
    if not ratings:
        return [0] * 6  # Return zeros if no ratings
    
    num_ratings = len(ratings)
    max_score = max(ratings, default=0)
    min_score = min(ratings, default=0)
    sum_score = sum(ratings)
    avg_score = sum_score / num_ratings if num_ratings > 0 else 0
    #var_score = np.var(ratings) if num_ratings > 0 else 0
    #return [num_ratings, max_score, min_score, sum_score, avg_score, var_score]
    std_dev = np.std(ratings) if ratings else 0
    return [num_ratings, max_score, min_score, sum_score, avg_score, std_dev]

    

dataDir = 'C:/Users/navne/Python Files/Data Acquisition BIA 627/HW5/'
file_name_test = dataDir + 'testTrack_hierarchy.txt'
file_name_train = dataDir + 'trainIdx2_matrix.txt'
output_file = dataDir + 'output2.csv'

train_data = {}
with open(file_name_train, 'r') as fTrain:
    for line in fTrain:
        userID, itemID, rating = line.strip().split('|')[:3]
        train_data.setdefault(userID, {})[itemID] = int(rating)

with open(file_name_test, 'r') as fTest, open(output_file, 'w', newline='') as fOut:
    csv_writer = csv.writer(fOut)
    csv_writer.writerow(['TrackID', 'Predictor'])

    for line in fTest:
        parts = line.strip().split('|')
        userID, trackID = parts[0], parts[1]
        kaggle_trackID = f"{userID}_{trackID}"

        albumID, artistID = parts[2], parts[3]
        genreIDs = parts[4:]

        # Initialize ratings list with album and artist ratings if they exist
        ratings = []
        if albumID in train_data.get(userID, {}):
            ratings.append(train_data[userID][albumID])
        if artistID in train_data.get(userID, {}):
            ratings.append(train_data[userID][artistID])

        # Append genre ratings to the ratings list
        for genreID in genreIDs:
            if genreID in train_data.get(userID, {}):
                ratings.append(train_data[userID][genreID])

        # Calculate stats for the ratings
        stats = calculate_stats(ratings)

        # Combine the statistics to form a total score for the track
        total_score = sum(stats)

        # Decision threshold to classify as '1' or '0'
        threshold = 150  # Adjust based on validation
        prediction = 1 if total_score > threshold else 0

        # Write the result to the CSV
        csv_writer.writerow([kaggle_trackID, prediction])
